In [0]:
dbutils.fs.ls("mnt/bronze/HumanResources/")

Out[1]: [FileInfo(path='dbfs:/mnt/bronze/HumanResources/Department/', name='Department/', size=0, modificationTime=1694698929000),
 FileInfo(path='dbfs:/mnt/bronze/HumanResources/Employee/', name='Employee/', size=0, modificationTime=1694698930000),
 FileInfo(path='dbfs:/mnt/bronze/HumanResources/EmployeeDepartmentHistory/', name='EmployeeDepartmentHistory/', size=0, modificationTime=1694698931000),
 FileInfo(path='dbfs:/mnt/bronze/HumanResources/EmployeePayHistory/', name='EmployeePayHistory/', size=0, modificationTime=1694698932000),
 FileInfo(path='dbfs:/mnt/bronze/HumanResources/JobCandidate/', name='JobCandidate/', size=0, modificationTime=1694698928000),
 FileInfo(path='dbfs:/mnt/bronze/HumanResources/Shift/', name='Shift/', size=0, modificationTime=1694698933000)]

In [0]:
dbutils.fs.ls("mnt/silver/")

Out[8]: []

In [0]:
table_name = []

for i in dbutils.fs.ls('mnt/bronze/HumanResources/'):
    table_name.append(i.name.split('/')[0])

In [0]:
from pyspark.sql. functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for i in table_name:
    path='/mnt/bronze/HumanResources/' + i + '/' + i + '.parquet'
    df = spark.read.format('parquet').load(path)
    column= df.columns

    for col in column:
        if "Date" in col or "date" in col:
            df = df.withColumn (col, date_format (from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))

    output_path='/mnt/silver/HumanResources/' +i +'/'
    df.write.format('delta').mode("overwrite").save(output_path)

In [0]:
display(df)

ShiftID,Name,StartTime,EndTime,ModifiedDate
1,Day,1970-01-01T07:00:00.000+0000,1970-01-01T15:00:00.000+0000,2008-04-30
2,Evening,1970-01-01T15:00:00.000+0000,1970-01-01T23:00:00.000+0000,2008-04-30
3,Night,1970-01-01T23:00:00.000+0000,1970-01-01T07:00:00.000+0000,2008-04-30
